In [79]:
import wntr  
import numpy as np 
import pandas as pd
import matplotlib as mpl
from matplotlib import figure
import matplotlib.pyplot as plt
import timeit 
import re

In [80]:
 # Replace with appropriate path and filename
directory='/Users/omaraliamer/Desktop/UofT/Papers/Clean Network Files/SWMM/Final Files/'
filename='N3a_4hr.inp'
name_only=filename[0:-4]
print("Selected File: ",name_only)
abs_path=directory+filename

Selected File:  N3a_4hr


In [81]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for node in network.junctions():

    # For all nodes that have non-zero demands
    if node[1].base_demand != 0:
        # Record node ID (name) and its desired demand (base_demand) in CMS
        demand_nodes.append(node[1].name)
        desired_demands.append(node[1].base_demand)
        elevations.append(node[1].elevation)

# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

In [87]:
tankids=['TankforNode'+str(id) for id in demand_nodes]
volumes=[demand* 60 * supply_duration for demand in desired_demands]
diameters=[round(np.sqrt(volume * 4 / np.pi),4) for volume in volumes]
zeros=[0.0000] *len(tankids)
MaxLevel=[1.0000]*len(tankids)
VolCurve=['    ']*len(tankids)
semicolons=[';']*len(tankids)
tanks_section=pd.DataFrame(list(zip(tankids,elevations,zeros,zeros,MaxLevel,diameters,zeros,VolCurve,semicolons)))
tanks_section=tanks_section.to_string(header=False,index=False,col_space=10).splitlines()

In [91]:
file=open(abs_path,'r')
lines=[]
linecount=0
found=0
for line in file:
    if re.search('\[TANKS\]',line):
        found=linecount+2
    linecount+=1
    lines.append(line)
file.close()

lines[found:found]=tanks_section

file=open(directory+name_only+'_STM_Generated.inp','w')
c=0
for line in lines:
    if c>=found and c<=found+len(tanks_section):
        file.write(line+'\n')
    else: file.write(line)    
    c+=1
file.close()
